# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [102]:
import pandas as pd
import os
import numpy as np
import boto3
import io
import sagemaker
from sklearn.metrics import accuracy_score, confusion_matrix

In [103]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [104]:
# set name of directory to save features data
data_dir = 'plagiarism_data'

# set prefix
prefix = 'sagemaker/plagiarism_detector'

# set output path for training artifacts
output_path='s3://{}/{}/'.format(bucket, prefix)

In [105]:
# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-634631323234/sagemaker/plagiarism_detector


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [106]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

sagemaker-pytorch-2021-01-16-17-15-34-011/source/sourcedir.tar.gz
sagemaker-record-sets/LinearLearner-2021-01-16-15-21-48-992/.amazon.manifest
sagemaker-record-sets/LinearLearner-2021-01-16-15-21-48-992/matrix_0.pbr
sagemaker-record-sets/LinearLearner-2021-01-16-15-49-45-364/.amazon.manifest
sagemaker-record-sets/LinearLearner-2021-01-16-15-49-45-364/matrix_0.pbr
sagemaker/plagiarism_detector/linear-learner-2021-01-16-15-22-14-994/output/model.tar.gz
sagemaker/plagiarism_detector/linear-learner-2021-01-16-15-22-14-994/profiler-output/framework/training_job_end.ts
sagemaker/plagiarism_detector/linear-learner-2021-01-16-15-22-14-994/profiler-output/system/incremental/2021011615/1610810700.algo-1.json
sagemaker/plagiarism_detector/linear-learner-2021-01-16-15-22-14-994/profiler-output/system/training_job_end.ts
sagemaker/plagiarism_detector/sagemaker-pytorch-2021-01-16-17-15-34-011/profiler-output/system/incremental/2021011617/1610817420.algo-1.json
sagemaker/plagiarism_detector/sagemaker

## Load data from S3

In [107]:
s3client = boto3.client('s3')

list_datasets=[]
for file in ['train.csv', 'test.csv']:
    data_body = s3client.get_object(Bucket=bucket, Key=os.path.join(prefix,file))['Body'].read()
    data_stream = io.BytesIO(data_body)
    df = pd.read_csv(data_stream, header=None, delimiter=",")
    list_datasets.append(df)

In [108]:
# load train data
train_data = list_datasets[0]
train_labels = train_data.iloc[:,0].values.astype('float32')
train_features = train_data.iloc[:,1:].values.astype('float32')

In [109]:
# load test data
test_data = list_datasets[1]
test_labels = test_data.iloc[:,0].values.astype('float32')
test_features = test_data.iloc[:,1:].values.astype('float32')

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [110]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

### Scikit-learn

In [150]:
# import Scikit-learn wrapper
from sagemaker.sklearn.estimator import SKLearn

# instantiate Scitkit-learn Random Forest Classifier estimator
sklearn_estimator = SKLearn(entry_point='train.py',
                            source_dir='source_sklearn',
                            role=role,
                            framework_version='0.23-1',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.c4.xlarge',
                            output_path=output_path,
                            sagemaker_session=sagemaker_session,
                            hyperparameters={
                                'n_estimators': 20,
                                #'max_features': 'auto',
                                'max_depth': 5,
                                'min_samples_split': 2 
                            })

### PyTorch

In [124]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# instantiate a pytorch estimator
pytorch_estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch',
                    role=role,
                    framework_version='1.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': 2,
                        'hidden_dim': 20,
                        'output_dim': 1,
                        'epochs': 80
                    })

### LinearLearner

In [112]:
# import LinearLearner
from sagemaker import LinearLearner

# instantiate LinearLearner
linear_estimator = LinearLearner(role=role,
                                instance_count=1,
                                instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

### Scikit-learn

In [151]:
%%time 
sklearn_estimator.fit({'train': input_data})

2021-01-16 18:42:48 Starting - Starting the training job...
2021-01-16 18:43:11 Starting - Launching requested ML instancesProfilerReport-1610822567: InProgress
......
2021-01-16 18:44:12 Starting - Preparing the instances for training......
2021-01-16 18:45:12 Downloading - Downloading input data...
2021-01-16 18:45:36 Training - Downloading the training image..2021-01-16 18:45:57,629 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-01-16 18:45:57,631 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-01-16 18:45:57,642 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-01-16 18:45:58,062 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-01-16 18:45:58,075 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-01-16 18:45:58,088 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus install

### PyTorch

In [125]:
%%time 
# train the estimator on S3 training data
pytorch_estimator.fit({'train': input_data})

2021-01-16 18:01:05 Starting - Starting the training job...
2021-01-16 18:01:32 Starting - Launching requested ML instancesProfilerReport-1610820064: InProgress
......
2021-01-16 18:02:32 Starting - Preparing the instances for training......
2021-01-16 18:03:33 Downloading - Downloading input data
2021-01-16 18:03:33 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-16 18:03:46,823 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-16 18:03:46,825 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-01-16 18:03:46,837 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-16 18:03:46,839 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-16 18:03:47,187 sagemaker-containers INFO     Module train does not provide a setup.py. 
Gener


2021-01-16 18:04:03 Uploading - Uploading generated training model
2021-01-16 18:04:03 Completed - Training job completed
Training seconds: 45
Billable seconds: 45
CPU times: user 538 ms, sys: 5.87 ms, total: 544 ms
Wall time: 3min 12s


### LinearLearner

In [42]:
# create record_set for LinearLearner
formatted_train_data = linear_estimator.record_set(train_features, labels=train_labels)

In [22]:
%%time

# Train your estimator on training data
linear_estimator.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-01-16 15:22:15 Starting - Starting the training job...
2021-01-16 15:22:38 Starting - Launching requested ML instancesProfilerReport-1610810534: InProgress
......
2021-01-16 15:23:39 Starting - Preparing the instances for training.........
2021-01-16 15:25:13 Downloading - Downloading input data
2021-01-16 15:25:13 Training - Downloading the training image...
2021-01-16 15:25:44 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[01/16/2021 15:25:38 INFO 140646578386752] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'qu


2021-01-16 15:26:01 Completed - Training job completed
Training seconds: 51
Billable seconds: 51
CPU times: user 657 ms, sys: 43.8 ms, total: 700 ms
Wall time: 4min 13s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

### Scikit-learn

In [152]:
%%time

sklearn_predictor = sklearn_estimator.deploy(initial_instance_count=1,
                                              instance_type='ml.t2.medium')

-------------------!CPU times: user 363 ms, sys: 21 ms, total: 384 ms
Wall time: 9min 32s


### PyTorch

In [126]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=pytorch_estimator.model_data,
                            role=role,
                            entry_point='predict.py',
                            framework_version='1.0',
                            py_version='py3',
                            source_dir='source_pytorch')

CPU times: user 13.5 ms, sys: 3.84 ms, total: 17.3 ms
Wall time: 74.1 ms


In [128]:
%%time
pytorch_predictor = model.deploy(initial_instance_count=1,
                                instance_type='ml.t2.medium')

---------------!CPU times: user 407 ms, sys: 12.5 ms, total: 420 ms
Wall time: 7min 32s


### LinearLearner

In [43]:
%%time 
linear_predictor = linear_estimator.deploy(initial_instance_count=1,
                                          instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------!CPU times: user 334 ms, sys: 27.7 ms, total: 361 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

### Scikit-learn

In [154]:
# First: generate predicted, class labels
test_y_preds = sklearn_predictor.predict(test_features)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [155]:
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y,test_y_preds)

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [156]:
print(pd.crosstab(test_y, test_y_preds, rownames=['actual (row)'], colnames=['prediction (col)']))

prediction (col)   0   1
actual (row)            
0                 10   0
1                  0  15


### PyTorch

In [137]:
# First: generate predicted, class labels
predictions = pytorch_predictor.predict(test_features)
test_y_preds = predictions.flatten()


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [138]:
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y,test_y_preds)

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.6

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [139]:
print(pd.crosstab(test_y, test_y_preds, rownames=['actual (row)'], colnames=['prediction (col)']))

prediction (col)  1.0
actual (row)         
0                  10
1                  15


### LinearLearner

In [92]:
# First: generate predicted, class labels
predictions = linear_predictor.predict(test_features)
test_y_preds = np.array([int(pred.label['predicted_label'].float32_tensor.values[0]) for pred in predictions])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [93]:
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y,test_y_preds)

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [94]:
print(pd.crosstab(test_y, test_y_preds, rownames=['actual (row)'], colnames=['prediction (col)']))

prediction (col)   0   1
actual (row)            
0                 10   0
1                  0  15


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 


### Question 2: How did you decide on the type of model to use? 

** Answer**:



----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [157]:
# uncomment and fill in the line below!
sklearn_predictor.delete_endpoint()
pytorch_predictor.delete_endpoint()
linear_predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [158]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '8Z5T4J7HDKEWAWAT',
   'HostId': 'iXzP+VpqEwLW6WVNXSFdTwsMz6fgnXULk7VReKMhXA/Eh1G7A/4n5+yN+rXOVxKxwOTs7Q/+iEU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'iXzP+VpqEwLW6WVNXSFdTwsMz6fgnXULk7VReKMhXA/Eh1G7A/4n5+yN+rXOVxKxwOTs7Q/+iEU=',
    'x-amz-request-id': '8Z5T4J7HDKEWAWAT',
    'date': 'Sat, 16 Jan 2021 18:58:00 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/plagiarism_detector/sagemaker-pytorch-2021-01-16-17-41-48-702/profiler-output/system/incremental/2021011617/1610819040.algo-1.json'},
   {'Key': 'sagemaker/plagiarism_detector/sagemaker-scikit-learn-2021-01-16-18-42-47-850/rule-output/ProfilerReport-1610822567/profiler-output/profiler-reports/MaxInitializationTime.json'},
   {'Key': 'sagemaker/plagiarism_detector/sagemaker-pytorch-2021-01-16-18-01-04-713/profiler-output/system/traini

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!